In [1]:
!pip install torch imgaug

     -------------------------------------- 162.5/162.5 MB 4.9 MB/s eta 0:00:00
     ------------------------------------- 948.0/948.0 kB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 8.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import subprocess
import zipfile
import numpy as np
import torch
import uptrain

from helper_files import read_json, KpsDataset
from helper_files import body_length_signal, plot_all_cluster

In [5]:
data_dir = "data"
remote_url = "https://oodles-dev-training-data.s3.amazonaws.com/data.zip"
orig_training_file = 'data/training_data.json'
if not os.path.exists(data_dir):
    print("Starting to download example dataset...")
    try:
        # Most Linux distributions have Wget installed by default.
        # Below command is to install wget for MacOS
        wget_installed_ok = subprocess.call("brew install wget", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
        print("Successfully installed wget")
    except:
        dummy = 1
    try:
        if not os.path.exists("data.zip"):
            file_downloaded_ok = subprocess.call("wget " + remote_url, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
            print("Data downloaded")
        with zipfile.ZipFile("data.zip", 'r') as zip_ref:
            zip_ref.extractall("./")
        print("Prepared Example Dataset")
        os.remove("data.zip")
    except Exception as e:
        print(e)
        print("Could not load training data")
        print("Please follow following steps to manually download data")
        print("Step 1: Paste the link https://oodles-dev-training-data.s3.amazonaws.com/data.zip in your browser")
        print("Step 2: Once the zip file is downloaded, unzip it here (i.e. YOUR_LOC/uptrain/examples/1_orientation_classification/")
else:
    print("Example dataset already present")

Example dataset already present


In [6]:
# Training data
training_file = 'data/training_data.json'

# A testing dataset to evaluate model performance
golden_testing_file = 'data/golden_testing_data.json'

# The data-points which the models sees in production
real_world_test_cases = 'data/real_world_testing_data.json'

# To annotate the collected data points, we extract the ground truth from a master annotation file 
# (we can also import from any other annotation pipelines such as an annotation job on Mechanical turk).
annotation_args = {'master_file': 'data/master_annotation_data.json'}

# Let's visualize some of the training examples
training_data = read_json(training_file, dataframe=True)
training_data.head()

,id,gt,Nose_X,Nose_Y,Left_Eye_X,Left_Eye_Y,Right_Eye_X,Right_Eye_Y,Left_Ear_X,Left_Ear_Y,...,Right_Hip_X,Right_Hip_Y,Left_Knee_X,Left_Knee_Y,Right_Knee_X,Right_Knee_Y,Left_Ankle_X,Left_Ankle_Y,Right_Ankle_X,Right_Ankle_Y
0,12002202098,1,536.832198,175.195562,275.183835,299.200492,321.438219,53.586762,438.831630,196.469141,...,355.320493,352.771307,282.603256,245.372110,-71.727270,148.854694,377.204408,118.812093,368.630415,32.683708
1,11004300163,1,127.299427,213.304615,120.006179,213.098156,119.749502,220.877945,123.240242,213.496218,...,282.739062,217.173672,331.887746,122.399542,369.934428,153.331971,407.764187,54.115847,373.738996,228.353284
2,18100208242,0,342.735809,75.173884,347.639763,69.066162,337.646757,69.463328,354.332650,70.183796,...,327.693125,175.678139,373.687116,236.403840,330.044019,238.719585,383.867725,289.895923,335.013380,292.018822
3,18100204340,0,335.262419,161.525842,340.358734,155.666068,329.570863,154.843509,347.034712,154.827196,...,316.727216,218.696415,375.741803,250.993843,285.306140,252.504808,384.364024,298.382275,275.523553,298.044853
4,12100104261,0,255.488160,62.442478,256.765451,58.603055,251.489867,58.323107,255.327531,55.634296,...,235.763698,136.902009,277.832912,177.679435,229.566259,191.052507,284.397434,213.477405,224.158806,239.154790


In [7]:
from helper_files import get_accuracy_torch, train_model_torch, BinaryClassification
train_model_torch(training_file, 'version_0')

Training on:  data/training_data.json  which has  1000  data-points
Epoch 0: Loss 6.108190503331922
Epoch 1: Loss 1.543215388148148
Epoch 2: Loss 0.9718632487262855
Epoch 3: Loss 0.7866230373424075
Epoch 4: Loss 0.5675198829147949
Epoch 5: Loss 0.5176476627772758
Epoch 6: Loss 0.3886559758861546
Epoch 7: Loss 0.4058310666466952
Epoch 8: Loss 0.2512015374009751
Epoch 9: Loss 0.3784226498949533
Model saved at:  trained_models_torch/version_0


In [8]:
get_accuracy_torch(golden_testing_file, 'version_0')

Evaluating model: version_0  on  15731  data-points


0.9289937066937893

In [10]:
checks = [{
        'type': uptrain.Anomaly.DATA_DRIFT,
        'reference_dataset': orig_training_file,
        'is_embedding': True,
        "measurable_args": {
            'type': uptrain.MeasurableType.INPUT_FEATURE,
            'feature_name': 'kps'   # keypoints
        },
    }]

print("Data Drift checks: ", checks)

Data Drift checks:  [{'type': <Anomaly.DATA_DRIFT: 'data_drift'>, 'reference_dataset': 'data/training_data.json', 'is_embedding': True, 'measurable_args': {'type': <MeasurableType.INPUT_FEATURE: 'input_feature'>, 'feature_name': 'kps'}}]


In [11]:
# Define the training pipeline to annotate collected edge cases and retrain the model automatically
training_args = {
    "annotation_method": {"method": uptrain.AnnotationMethod.MASTER_FILE, "args": annotation_args}, 
    "training_func": train_model_torch, 
    "orig_training_file": orig_training_file,
}

# Define evaluation pipeline to test retrained model against original model
evaluation_args = {
    "inference_func": get_accuracy_torch,
    "golden_testing_dataset": golden_testing_file,
}

print("Training Pipelines: ", training_args, "\n")
print("Evaluation Pipelines: ", evaluation_args)

Training Pipelines:  {'annotation_method': {'method': <AnnotationMethod.MASTER_FILE: 1>, 'args': {'master_file': 'data/master_annotation_data.json'}}, 'training_func': <function train_model_torch at 0x000002C94DB11430>, 'orig_training_file': 'data/training_data.json'} 

Evaluation Pipelines:  {'inference_func': <function get_accuracy_torch at 0x000002C94DB111F0>, 'golden_testing_dataset': 'data/golden_testing_data.json'}


In [12]:
cfg = {
    "checks": checks, 
    "training_args": training_args,
    "evaluation_args": evaluation_args,

    # Retrain when 200 datapoints are collected in the retraining dataset
    "retrain_after": 200,
    
    # A local folder to store the retraining dataset
    "retraining_folder": "uptrain_smart_data",
    
    # A function to visualize clusters in the data
    "cluster_visualize_func": plot_all_cluster,
}

# Initialize the UpTrain framework object with config 
framework = uptrain.Framework(cfg)
print("Successfully Initialized UpTrain Framework")

Successfully Initialized UpTrain Framework


In [13]:
inference_batch_size = 16
model_dir = 'trained_models_torch/'
model_save_name = 'version_0'
real_world_dataset = KpsDataset(
    real_world_test_cases, batch_size=inference_batch_size, is_test=True
)
model = BinaryClassification()
model.load_state_dict(torch.load(model_dir + model_save_name))
model.eval()

for i,elem in enumerate(real_world_dataset):

    # Do model prediction
    inputs = {"data": {"kps": elem[0]["kps"]}, "id": elem[0]["id"]}
    x_test = torch.tensor(inputs["data"]["kps"]).type(torch.float)
    test_logits = model(x_test).squeeze() 
    preds = torch.round(torch.sigmoid(test_logits)).detach().numpy()

    # Log model inputs and outputs to the uptrain Framework to monitor data drift
    idens = framework.log(inputs=inputs, outputs=preds)

    # Retrain only once
    if framework.version > 1:
        break

50  edge cases identified out of  13424  total samples
101  edge cases identified out of  15792  total samples
150  edge cases identified out of  24128  total samples
200  edge cases identified out of  26608  total samples

Kicking off re-training
Creating retraining dataset: uptrain_smart_data\1/training_dataset.json  by merging  data/training_data.json  and collected edge cases.

Training on:  uptrain_smart_data\1/training_dataset.json  which has  2005  data-points
Epoch 0: Loss 6.547668935537338
Epoch 1: Loss 2.0966730518441907
Epoch 2: Loss 0.9921911038540454
Epoch 3: Loss 0.5552357879825879
Epoch 4: Loss 0.3871322223683595
Epoch 5: Loss 0.28291741295483325
Epoch 6: Loss 0.2265821895140971
Epoch 7: Loss 0.1783209576342085
Epoch 8: Loss 0.11531237676203318
Epoch 9: Loss 0.13162714021632768
Model saved at:  trained_models_torch/version_1
Model retraining done...

Generating comparison report...
Evaluating model: version_0  on  15731  data-points
Evaluating model: version_1  on  15731